In [1]:
import os
import pandas as pd
import numpy as np
import scipy as sp
from scipy import signal
from ml_time_series import generate_envelope
import timeit
from sklearn.externals import joblib

---
# Processing
---

In [26]:
def proccess_data(FILE_,DATA,NUM_SAMPLES,LABEL):

    Xc = np.genfromtxt('DB_GAIT/'+FILE_+'.txt', delimiter=",", usecols=(1,2,3,4))
    
    print '\nXc shape ', Xc.shape
    
    #Create temporal serie
    Xc = generate_envelope(Xc, NUM_SAMPLES)
    print 'Xc temporal-serie shape ', Xc.shape
    
    #Labeling the type of movement
    C = (np.ones(len(Xc))*LABEL).reshape((len(Xc),1))
    Xc = np.hstack((Xc.reshape(Xc.shape),C.reshape((len(Xc),1))))
    print 'Xc labeled shape ', Xc.shape
    
    # Salving in file on the folder <classifier_data>
    np.save('./preprocessed_data/'+FILE_, Xc, allow_pickle=False)
    print FILE_+'.npy'
    
    DATA.append(FILE_+'.npy')

In [27]:
files_w = ['data_w_1', 'data_w_2', 'data_w_3', 'data_w_4', 'data_w_5', 'data_w_6', 'data_w_7', 'data_w_8', 'data_w_9', 
           'data_w_10', 'data_w_11', 'data_w_12']
files_r = ['data_r_1', 'data_r_2', 'data_r_3', 'data_r_4', 'data_r_5', 'data_r_6', 'data_r_7', 'data_r_8', 'data_r_9', 
           'data_r_10', 'data_r_11', 'data_r_12']
files_us = ['data_us_1', 'data_us_2', 'data_us_3', 'data_us_4', 'data_us_5', 'data_us_6', 'data_us_7', 'data_us_8', 
            'data_us_9', 'data_us_10', 'data_us_11', 'data_us_12']
files_ds = ['data_ds_1', 'data_ds_2', 'data_ds_3', 'data_ds_4', 'data_ds_5', 'data_ds_6', 'data_ds_7', 'data_ds_8', 
            'data_ds_9', 'data_ds_10', 'data_ds_11', 'data_ds_12']

NUM_SAMPLES = 500

DATA = []

start = timeit.default_timer()

for i in files_w:
    proccess_data(i,DATA,NUM_SAMPLES,0)

for i in files_r:
    proccess_data(i,DATA,NUM_SAMPLES,1)

for i in files_us:
    proccess_data(i,DATA,NUM_SAMPLES,2)

for i in files_ds:
    proccess_data(i,DATA,NUM_SAMPLES,3)

print DATA

stop = timeit.default_timer()

ti = stop - start 
print ('Tempo total para o envelopamento: %.2f min' % (ti/60))


Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled shape  (4502, 2001)
data_w_1.npy

Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled shape  (4502, 2001)
data_w_2.npy

Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled shape  (4502, 2001)
data_w_3.npy

Xc shape  (6001, 4)
Xc temporal-serie shape  (5502, 2000)
Xc labeled shape  (5502, 2001)
data_w_4.npy

Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled shape  (4502, 2001)
data_w_5.npy

Xc shape  (6001, 4)
Xc temporal-serie shape  (5502, 2000)
Xc labeled shape  (5502, 2001)
data_w_6.npy

Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled shape  (4502, 2001)
data_w_7.npy

Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled shape  (4502, 2001)
data_w_8.npy

Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled shape  (4502, 2001)
data_w_9.npy

Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled sha

In [38]:
%%bash
paplay /usr/share/sounds/freedesktop/stereo/complete.oga

['data_w_1.npy', 'data_w_2.npy', 'data_w_3.npy', 'data_w_4.npy', 'data_w_5.npy', 'data_w_6.npy', 'data_w_7.npy', 'data_w_8.npy', 'data_w_9.npy', 'data_w_10.npy', 'data_w_11.npy', 'data_w_12.npy', 'data_r_1.npy', 'data_r_2.npy', 'data_r_3.npy', 'data_r_4.npy', 'data_r_5.npy', 'data_r_6.npy', 'data_r_7.npy', 'data_r_8.npy', 'data_r_9.npy', 'data_r_10.npy', 'data_r_11.npy', 'data_r_12.npy', 'data_us_1.npy', 'data_us_2.npy', 'data_us_3.npy', 'data_us_4.npy', 'data_us_5.npy', 'data_us_6.npy', 'data_us_7.npy', 'data_us_8.npy', 'data_us_9.npy', 'data_us_10.npy', 'data_us_11.npy', 'data_us_12.npy', 'data_ds_1.npy', 'data_ds_2.npy', 'data_ds_3.npy', 'data_ds_4.npy', 'data_ds_5.npy', 'data_ds_6.npy', 'data_ds_7.npy', 'data_ds_8.npy', 'data_ds_9.npy', 'data_ds_10.npy', 'data_ds_11.npy', 'data_ds_12.npy']

In [2]:
DATA = ['data_w_1.npy', 'data_w_2.npy', 'data_w_3.npy', 'data_w_4.npy', 'data_w_5.npy', 'data_w_6.npy', 
        'data_w_7.npy',  
        'data_r_1.npy', 'data_r_2.npy', 'data_r_3.npy', 'data_r_4.npy', 'data_r_5.npy', 'data_r_6.npy', 
        'data_r_7.npy', 
        'data_us_1.npy', 'data_us_2.npy', 'data_us_3.npy', 'data_us_4.npy', 'data_us_5.npy', 'data_us_6.npy', 
        'data_us_7.npy',  
        'data_ds_1.npy', 'data_ds_2.npy', 'data_ds_3.npy', 'data_ds_4.npy', 'data_ds_5.npy', 'data_ds_6.npy', 
        'data_ds_7.npy']

len(DATA)

28

In [2]:
DATA = ['data_w_1.npy', 'data_w_2.npy', 'data_w_3.npy', 'data_w_4.npy', 'data_w_5.npy', 'data_w_6.npy', 'data_w_7.npy', 'data_w_8.npy', 'data_w_9.npy', 'data_w_10.npy', 'data_w_11.npy', 'data_w_12.npy', 'data_r_1.npy', 'data_r_2.npy', 'data_r_3.npy', 'data_r_4.npy', 'data_r_5.npy', 'data_r_6.npy', 'data_r_7.npy', 'data_r_8.npy', 'data_r_9.npy', 'data_r_10.npy', 'data_r_11.npy', 'data_r_12.npy', 'data_us_1.npy', 'data_us_2.npy', 'data_us_3.npy', 'data_us_4.npy', 'data_us_5.npy', 'data_us_6.npy', 'data_us_7.npy', 'data_us_8.npy', 'data_us_9.npy', 'data_us_10.npy', 'data_us_11.npy', 'data_us_12.npy', 'data_ds_1.npy', 'data_ds_2.npy', 'data_ds_3.npy', 'data_ds_4.npy', 'data_ds_5.npy', 'data_ds_6.npy', 'data_ds_7.npy', 'data_ds_8.npy', 'data_ds_9.npy', 'data_ds_10.npy', 'data_ds_11.npy', 'data_ds_12.npy']


In [3]:
start = timeit.default_timer()

Xc = np.load("preprocessed_data/"+DATA[0])
for i in DATA[1:]:
    Xc = np.vstack((Xc,np.load("preprocessed_data/"+i))) 
    
stop = timeit.default_timer()
ti = stop - start 
print ('Tempo total para o envelopamento: %.2f min' % (ti/60))
Xc.shape

Tempo total para o envelopamento: 0.54 min


(231096, 2001)

save_it_now(Xc, './preprocessed_data/vstack1.data')

start = timeit.default_timer()

for i in DATA[30:]:
    Xc = np.vstack((Xc,load_file("preprocessed_data/"+i))) 
    
stop = timeit.default_timer()
ti = stop - start 
print ('Tempo total para o envelopamento: %.2f min' % (ti/60))

Xc.shape

In [4]:
Xc.shape

(231096, 2001)

In [5]:
X = Xc[:,0:(Xc.shape[1]-2)]

yz = Xc[:,[(Xc.shape[1]-1)]]
y = np.array([])
for i in range(len(yz)):
    y = np.hstack((y,yz[i]))

X.shape, y.shape

((231096, 1999), (231096,))

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=10)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [9]:
print X_train_std.shape

(161767, 1999)


# Random Forest Classifier

In [10]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, n_jobs = 4)

print 'Training time'
%time rfc = rfc.fit(X_train_std, y_train)

print''

print 'Predict time'
%time y_pred = rfc.predict(X_test_std)

print''

from sklearn.metrics import accuracy_score
print ('ClassifyRF accuracy:---------->%.2f %%' % (accuracy_score(y_test, y_pred)*100))

Training time
CPU times: user 27min 17s, sys: 1.43 s, total: 27min 19s
Wall time: 6min 59s

Predict time
CPU times: user 13.1 s, sys: 396 ms, total: 13.5 s
Wall time: 5.6 s

ClassifyRF accuracy:---------->94.85 %


In [11]:
#Save the classifier to file
joblib.dump(rfc, 'rfc.pkl') 


['rfc.pkl']

In [7]:
#Load the classifier from file
rfc = joblib.load('rfc.pkl')

## Validation

In [34]:
DATAV = ['data_w_9.npy', 'data_r_9.npy', 'data_us_9.npy', 'data_ds_9.npy']

Xc = np.load("preprocessed_data/"+DATAV[0])
for i in DATAV[1:]:
    Xc = np.vstack((Xc,np.load("preprocessed_data/"+i)))
    
Xvalid = Xc[:,0:(Xc.shape[1]-2)]

yz = Xc[:,[(Xc.shape[1]-1)]]
yvalid = np.array([])
for i in range(len(yz)):
    yvalid = np.hstack((yvalid,yz[i]))

print Xvalid.shape, yvalid.shape

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#sc.fit(Xvalid)
sc.fit(X_train)
X_valid_std = sc.transform(Xvalid)

print X_valid_std.shape

(18008, 1999) (18008,)
(18008, 1999)


In [35]:
#Validation
print 'Predict time'
%time y_pred = rfc.predict(X_valid_std)

print''

from sklearn.metrics import accuracy_score

result = ('ClassifyRF accuracy:---------->%.2f %%' % (accuracy_score(y_pred, yvalid)*100))

print result

Predict time
CPU times: user 1.94 s, sys: 76 ms, total: 2.01 s
Wall time: 741 ms

ClassifyRF accuracy:---------->36.00 %


ClassifyRF 8  accuracy:---------->36.00 %
ClassifyRF 9  accuracy:---------->50.07 %
ClassifyRF 10 accuracy:---------->60.37 %
ClassifyRF 11 accuracy:---------->59.10 %
ClassifyRF 12 accuracy:---------->50.10 %